# Inteiros e números decimais (Float)

***DISCLAIMER: Este notebook foi escrito com base no que li [neste](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/) capítulo do manual***

Bem aqui lembramos , por exemplo, que números representados no nosso código são *numeric literals*: 1 e 1.0 são numeric literals.
Mas as representações como objetos desses mesmos números na memória, são chamados de *numeric primitives*.

Aqui temos as primitivas numéricas (prefiro *numeric primitives*) presentes em Julia:
* Inteiros
    * Int8, Int16, Int32, Int64, Int128
    * UInt8, UInt16, UInt32, UInt64, UInt128
    * Bool

* Decimais
    * Float16 (half precision), Float32(single precision), Float64(double precision)
    

Lembrando que U vem de *<u>unsigned</u>*, o que significa que são primitivas onde só permitem números sem sinal, ou seja, ***números positivos***.

 O `typeof` diz-nos o tipo de uma variável/constante/valor, e caso este seja inteiro, a primitiva pode mudar consoante a arquitetura do vosso computador (32 ou 64 bits).

In [63]:
# No meu caso dará Int64, pois tenho um CPU de 64 bits
typeof(1)

Int64

Pode-se obter a informação do número de bits usados pelo CPU, utilizando o `Sys.WORD_SIZE`

In [64]:
Sys.WORD_SIZE

64

Também se pode usar os *types* `UInt` e `Int` assim como as restantes primitivas citadas acima. Eles podem inclusive, modificar o tipo das variáveis

In [65]:
a = UInt8(245)
println( typeof(a) )

a = UInt(a)
println( typeof(a) )

UInt8
UInt64


***Nota***: Inteiros que não possa ser representados em 32 bits, são convertidos para 64 bits, independemente da arquitetura.


Algo também bastante interessante é que números em representação hexadecimal, octal e binário são sempre representados por primitivas *unsigned*, isto deve-se ao facto de que números negativos são convertidos para uma representação binária [Complemento para dois](https://pt.wikipedia.org/wiki/Complemento_para_dois).

A quantidade de números 1 à esquerda, dita o tamanho mínimo necessário para armazenar o número. 
Por exemplo 0xff pode ser armazenado em `UInt8`, porém 0xfff já será armazenado em `UInt16`.

In [66]:
println("1: ", typeof( 1 ) )
println("-1: ", typeof( -1 ) )
println("0x1: ", typeof( 0x1 ) )
println("-0x1: ", typeof( -0xfff ) )
-0x1

1: Int64
-1: Int64
0x1: UInt8
-0x1: UInt16


0xff

Caso queiramos saber o número mínimo e máximo que uma primitiva pode representar, basta-nos usar as funções `typemin` e `typemax`

In [67]:
typemin(Int16)

-32768

In [68]:
typemax(Int16)

32767

## Overflow

Quando tentamos fazer um overflow (armazenar um valor tão grande/pequeno que nenhuma primitiva consegue representar), Julia vai aplicar [aritmética modular](https://en.wikipedia.org/wiki/Modular_arithmetic) fazendo com que os valor volte para o intervalo da primitiva.

Caso queiramos pode representar esse número sem usar este 'wraparound', devemos fazer uso do `BigInt`

In [69]:
# Integer Overflow (64bits)
x = typemax(Int64)

9223372036854775807

In [70]:
# Vai transformá-lo no extremo negativo
x + 1 == typemin(Int64)

true

In [71]:
# Contornar esse efeito com o uso do BigInt
x = BigInt(typemax(Int64))
x + 1

9223372036854775808

## Divisões

As divisões também são bem interessantes por aqui. Todos sabemos que não podemos dividir por 0 e se usarmos a função `div` ela retornará um erro de divisão, assim como a divisão do menor número possível por -1, como podemos ver abaixo.

In [72]:
# Dividir 1 por 0
div(1, 0)

LoadError: DivideError: integer division error

In [73]:
# Dividir o typemin de u por -1
div(typemin(Int64), -1)

LoadError: DivideError: integer division error

Porém se dividirmos um número por 0 sem o uso da função `div`, vamos obter um valor infinito (`Inf`) do tipo `Float`.

Para a divisão do menor número negativo por -1 dará um valor próximo do maior número possível.

In [74]:
println("DIVIDIR NÚMERO POR 0")
println( 1/0 )
println( typeof( 1/0 ) )
println("\nDIVIDIR MENOR NÚMERO NEGATIVO POR -1")
println( typemin(Int64)/-1 ≈ typemax(Int64))
println( typeof( typemin(Int64)/-1 ) )

DIVIDIR NÚMERO POR 0
Inf
Float64

DIVIDIR MENOR NÚMERO NEGATIVO POR -1
true
Float64


Também temos as funções `rem` (resto) e `mod` (resto).

A diferença entre elas confude-me um pouco, sei que `rem` é o mesmo que `x % y`, onde o sinal do divisor (y, neste caso) não importa. Quando o dividendo e divisor têm o mesmo sinal, `rem` e `mod` dão o mesmo resultado.

Muitas coisas em Julia são iguais ao MATLAB e essas funções confirmam isso! Ao pesquisar um pouco, encontrei uma resposta que ajudou-me a elucidar um pouco.

Pensemos então na divisão de $\frac{5}{-3}$

<hr>


***Caso do `rem`***

Este é o resto normal como se fizéssemos a conta manualmente, ou seja, o resto será ***2***.

<hr>

***Caso do `mod`***
Este é um caso que está relacionado à [aritmética modular](https://en.wikipedia.org/wiki/Modular_arithmetic).

Pensemos nos múltiplos do nosso divisor, que no caso é 3 => $-3\mathbb{Z} = -3\{...,-3,-2,-1,0,1,2,3,...\} = \{...,9,6,3,0,-3,-6,-9,...\}$

O número que mais se proxima de 5 é o 6, logo temos de somar ***-1*** e esse será o nosso resto.

<hr>

A resposta completa está aqui (em inglês): [rem & mod](https://www.quora.com/I-am-a-beginner-in-MATLAB-programming-What-is-the-difference-between-the-two-operations-rem-and-mod-How-are-both-functionally-different)

<hr>

****Regra Geral****:

* Dividendo e divisor têm o mesmo sinal? Se sim, ambas vão retornar o mesmo.
* Para o `rem` só o sinal do dividendo muda o resultado final.

## Números decimais (Floats)

Assim como os hexadecimais são representados pela primitiva `UInt`, os números `Float` que tenham um 'f' são representados pela primitiva `Float32`.

***Nota***: O 'f' tem de estar entre os números decimais.

In [75]:
typeof(1.4f5)

Float32

Também podemos definir números em notação científica.

In [76]:
1.2e-4

0.00012

Floats com representações hexadecimais também são possíveis, basta colocarmos um 'p' entre os número decimais e começar o número com o prefixo que define os hexadecimais: '0x'.

In [77]:
0x1.8p3

12.0

Algo útil quando trabalhamos com números "grandes" é poder separar os dígitos, assim fica mais fácil distinguir as centenas, milhares, milhões etc. Podemos fazer isso utilizando underscores('_')

In [78]:
100_000

100000

Floats têm 2 zeros que significam o mesmo, porém na representação binária são representados de forma diferente. Isso deve-se ao que já referi: [Complemento para dois](https://pt.wikipedia.org/wiki/Complemento_para_dois)

In [79]:
# Os 2 zerosdo tipo Float 
0.0 == -0.0

true

In [80]:
# Representação do 0.0 em binário
bitstring(0.0)

"0000000000000000000000000000000000000000000000000000000000000000"

In [81]:
# Representação do -0.0 em binário
bitstring(-0.0)

"1000000000000000000000000000000000000000000000000000000000000000"

O máximo e mínimo dos floats são -Inf e Inf, porém esses referem-se a `Float64`, as outras primitivas de Floats têm Infs com sufixo igual ao número de bits: `Inf32`, `Inf16`.

In [82]:
typemin(Float64)

-Inf

In [83]:
typemax(Float64)

Inf

### Machine Epsilon

Entre 2 números haverá uma quantidade de números decimais gigantesca! E apesar destas máquinas denominadas de computadores terem capacidades de cálculo impressionantes, elas têm as suas limitações.

Tendo isso em conta, criou-se este tal de *Machine Epsilon*. Ele, basicamente, calcula a distância entre 1.0 e o próximo maior decimal (isto, caso não passemos nenhum argumento). Se passarmos um número decimal como argumento, ele retornará o próximo decimal.

In [84]:
eps()

2.220446049250313e-16

In [85]:
eps(0e14)

5.0e-324

In [86]:
eps(1e-14)

1.5777218104420236e-30

In [87]:
eps(10.0)

1.7763568394002505e-15

***Podemos observar que quando mais próximos de zero estiver o nosso número decimal, maior a precisão do `Float`! Ou seja, o número a seguir ao nosso decimal estará muitoooo próximo.***

***Já se nos afastarmos de 0, verificamos que a precisão diminui significativamente.***

***Nota***: Os métodos `prevfloat` e `nextfloat` retornam os decimais mais próximos (antes e depois, respetivamente).

### Arredondamentos

Os arredondamentos são feitos sempre que não haja um representação precisa de um decimal. Utiliza-se então o método `RoundNearest`.

## Aritméticas com Precisão Arbitrária

Precisão arbitrária vem do tipo `BigInt`, `BigFloat` etc ... 
É arbitrária porque não é algo primitivo, é algo 'emulado' pela  linguagem Julia. Por isso é normal cálculos de `BigInt` e de outros tipos de precisão arbitrária, sejam bem mais lentos, do que os tipos primitivos que falei acima.


Podemos representar esses números como hexadecimais, octais, binários.

In [88]:
# Converter um número imenso em BigInt
x = BigInt( typemax(Int64) )+1

9223372036854775808

In [89]:
# Transformar uma string desse número em BigInt utilziando a macro/string_literal 'big'
x = big"9223372036854775808"

9223372036854775808

***Nota***: Para promover números para `BigInt` e `BigFloat`, precisa de ser de forma ***explícita***!

### Precisão do BigFloat

Podemos mudar a precisão deles <u>globalmente</u> ou <u>localmente</u>. Para isso fazemos uso de `setprecision` e `setrounding`.

Para fazê-lo localmente, temos de envolver tudo em um bloco `do`.

In [90]:
# Arredondamento local (arredondamento para cima)
setrounding(BigFloat, RoundUp) do
    BigFloat(1) + parse(BigFloat, "0.1")
end

1.100000000000000000000000000000000000000000000000000000000000000000000000000003

In [91]:
# Definir precisão localmente
setprecision(30) do
    BigFloat(3) + BigFloat(0.1)
end

3.1000000015

## Coeficientes

Em Python uma expressão matemática como esta: $2x^2$ - seria traduzida em `2 * x**2`.

Julia permite-nos colocar coeficientes nas nossas variáveis sem termos de explicitamente definir a multiplicação.

In [92]:
x = 3
2x^2

18

A precedência destes coeficientes são:
* Inferiores a operadores unários (como a negação): `-2x` == `(-2) * x`
* Mas igualam a precedência com unários quando têm expoente: `2x^3` == `2 * (x^3)`
* Têm precedência maior que `*`,`/` e `//`, logo: 6 // 2(2) == `6 / (2*2)`

In [93]:
# Expressões em parêntesis antes de uma variável, são levadas como coeficientes
x = 3
(x-1)x

6

***Atenção***

Expressões com justaposição como : $x(x+5)$ ou $x^3(x-10)$ - não funcionam e são consideradas como chamadas para alguma função, por exemplo, $x(x+5)$ para Julia estamos a chamar a função x passando `x+5` como argumento.

### Problemas com coeficientes

Estes coeficientes que funcionam por justposição são bastante úteis! E fazem todo o sentido em uma linguagem científica como Julia, mas como tudo na vida, tem as suas desvantagens.

Olhemos para esta representação hexadecimal: `0xff` - pensando no que vimos acima, não parece que 0 está a multiplicar uma variável xff ??


Isto pode causar conflitos e é por isso que temos de ter cuidado a nomear variáveis e também ponderar bem quando usar essa sintaxe de justaposição de coeficientes e variáveis!

## Zeros e Uns

Se nós quisermos obter um 0 ou um 1 representados por uma primitiva ou tipo de precisão arbitrária, podemos fazê-lo com `zero` e `one`.

In [94]:
one(BigInt)

1

In [95]:
zero(Float16)

Float16(0.0)